## TODO
rename file folders -- done 
check if differenet image count per class is problem -- 
load the dataset probably -- 
resize images via PIL Image load > convert to torch tensor or directly loading it via torchvision read image --


# Imports


In [137]:
import torch
#to load the dataset and to split the dataset
from torch.utils.data import DataLoader, random_split
#for resizing of images
from torchvision import transforms
#to read images from path
from torchvision.io import read_image
from torchvision.datasets import ImageFolder
from PIL import Image
from pathlib import Path
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [138]:
#simply getting working directory and appending the archive directories to it
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, "archive/raw-img")
print("Dataset directory: ", dataset_dir)

Dataset directory:  c:\Users\adamg\Desktop\Ulohy\3 vyska\ZNEUS\cvika\zadania\zadanie 2\archive/raw-img


In [139]:
#experiment, by which i determined that the average size of the images is 250x320 (width x height)so i will would resize it to that size but working with width=height is easier so the size would be 250x250, but from presentation of CNN we learned that resizing them to 224x224 could be better
# average_height = 0
# average_width = 0
# images_count = 0
# for file_name in os.listdir(dataset_dir):
#     if not os.path.isdir(os.path.join(dataset_dir, file_name)):
#         continue # Not a directory
#     class_dir = os.path.join(dataset_dir, file_name)
#     for class_file_name in os.listdir(class_dir):
#         img_size = Image.open(os.path.join(class_dir, class_file_name)).size
#         average_width += img_size[0]
#         average_height += img_size[1]
#         images_count += 1
#     print(file_name, "avrHei", average_height/images_count, "avrWid:", average_width/images_count)

# average_height = 0
# average_width = 0
# images_count = 0

#second experiment to see how the image can be converted and worked on with Image nad pytorch tensor transformation
# torch_img_size = (224, 224)
# img_path = os.path.join(dataset_dir, "butterfly")
# listDir = os.listdir(img_path)
# img_name = listDir[0]
# img_path = os.path.join(img_path, img_name)
# #toto nacita obrazok 
# torchImage = read_image(img_path) / 255.0

# second_torch_image = transform(torchImage)


In [ ]:
#size of the image that will be after "normalization" (setting all the images to the same size)
img_size = (224, 224)
#transormation that will be used on every image https://docs.pytorch.org/vision/stable/transforms.html
image_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor()
    ])
analysis_transform = transforms.ToTensor()

#loading the dataset with pytorch using ImageFolder https://docs.pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html

image_dataset = ImageFolder(
    root = dataset_dir, 
    transform = image_transform 
)



#if you see ragno, my translate_names.py is not working correctly xd (its spider)
print("Dataset classes: ", image_dataset.classes)

#splitting the dataset to 70% train, 10% validation and 20% train using random_split   https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.random_split

dataset_size = len(image_dataset)
print("Dataset Size:\t", dataset_size)
train_size = int(0.7 * dataset_size)
validation_size   = int(0.1 * dataset_size)
test_size  = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(image_dataset, [train_size, validation_size, test_size])
print("Split sizes:\nTrain:\t\t", len(train_dataset), "\nValidation:\t", len(validation_dataset), "\nTest:\t\t", len(test_dataset))


Dataset classes:  ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']
Dataset Size:	 26179
Split sizes:
Train:		 18325 
Validation:	 2617 
Test:		 5237


In [144]:
#loading the image_dataset to dataLoader https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
train_loader = DataLoader(
    dataset = train_dataset,
    shuffle = True, #this shuffles order of images each epoch, random_split randomizes it only once, maybe we dont need this
    batch_size = 10  #1 is default, maybe experiment with this later
)
validation_loader = DataLoader(
    dataset = validation_dataset,
    shuffle = True,
    batch_size = 10  
)
test_loader = DataLoader(
    dataset = test_dataset,
    shuffle = True,
    batch_size = 10  
)

#this is how we can iterate through the DataLoader
for image_tensor, labels in train_loader:
    #shape is [Batch_size, Channels, Height, Width]
    print(image_tensor.shape)
    #and label (correct class of the image) for each image in tensor (images)
    print(labels) 
    break

torch.Size([10, 3, 224, 224])
tensor([3, 8, 1, 8, 4, 4, 7, 4, 9, 2])


In [ ]:
#data anylisis here >
#loading dataset for data vizualization and analysis
analysis_dataset = ImageFolder(
    root = dataset_dir,
    transform = analysis_transform
)
analysis_loader = DataLoader(
    dataset = analysis_dataset,
    shuffle = True,
    batch_size = 10  
)


['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']
